In [2]:
import numpy as np
import pandas as pd

# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

import os
import sys
import shutil
import pickle
import tenseal as ts

from sklearn.datasets import load_breast_cancer

2022-03-05 17:09:57.455456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-05 17:09:57.455477: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


Read the data and check it

In [3]:
data = load_breast_cancer()
# target = 1 - data.target

# MUST reshape target (y) to be list of lists for TensorFlow 
target = data.target.reshape(-1,1)

In [4]:
temp = np.zeros(target.size)
target = np.insert(target, 0, temp, axis=1)
for i in range(len(target)):
  target[i][0] = 1 - target[i][1]

In [5]:
print(data.target.reshape(-1))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 

In [6]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data.data, target)

In [7]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [8]:
X_train[:3]

array([[-0.30650268,  0.62104272, -0.16828882, -0.36588129,  2.19035687,
         1.77664083,  1.27444525,  1.17841776,  2.02983185,  1.61676872,
        -0.33685784, -0.40786249, -0.20881921, -0.32773331, -0.44966412,
         0.59461344,  0.14631533,  0.10335768,  0.13164223,  0.02065071,
        -0.13857129,  0.8430959 , -0.00521991, -0.22513212,  1.66926677,
         1.83915393,  1.33954402,  1.44311687,  2.37234416,  1.29276721],
       [ 1.29198801, -0.46466634,  1.22185119,  1.18250508,  0.65188085,
         0.15325043,  0.7614691 ,  1.00020322,  0.6074009 , -0.06209409,
         1.06800303,  0.63523719,  0.98549316,  1.02784828,  0.36244732,
        -0.22544912,  0.07739708,  0.74161963,  0.0042158 ,  0.34936176,
         1.27241411, -0.09340583,  1.16718532,  1.19251002,  0.80945305,
        -0.12025427,  0.31168638,  0.84815064,  0.02745907,  0.07219191],
       [-0.85439525, -0.99366076, -0.87320053, -0.75936928, -1.31441794,
        -1.06273142, -0.75168296, -1.05662044, -0

In [9]:
# shape None == allow dynamic number of rows; X_train.shape[1] == number of features
X = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]), name='inputs')
y = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='targets')

In [10]:
# First hidden layer
# X == features coming in, 30 == number of features == neurons
h1 = tf.layers.dense(X, 30, name='dense1')
h1 = h1 * h1
# Second hidden layer
# h1 == features coming in, 26 == number of neurons == arbitrary
h2 = tf.layers.dense(h1, 16, name='dense2')
h2 = h2 * h2
# Last/output layar always has 1 neuron for binary classification problems
model1 = tf.layers.dense(h2, 2, name='dense3')

model = tf.sigmoid(model1, name='sigmoid')
# model = approx_sigmoid(model)

/tmp/ipykernel_5705/1031069605.py:3: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h1 = tf.layers.dense(X, 30, name='dense1')
/home/thuy/.local/lib/python3.8/site-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/tmp/ipykernel_5705/1031069605.py:7: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h2 = tf.layers.dense(h1, 16, name='dense2')
/tmp/ipykernel_5705/1031069605.py:10: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  model1 = tf.layers.dense(h2, 2, name='dense3')


In [11]:
loss = tf.losses.log_loss(y, model)

# Define training operation
training_op = tf.train.AdamOptimizer(.01).minimize(loss)

In [12]:
# Initialize
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    # Train model with 1000 epochs
    # saver.restore(sess, '/content/drive/MyDrive/Colab Notebooks/result/nn_data/net-1000')
    for epoch in range(1000):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        if (epoch % 100 == 0):
            training_loss = sess.run(loss, feed_dict={X: X_train, y: y_train})
            test_loss = sess.run(loss, feed_dict={X: X_test, y: y_test})
            print('epoch:',epoch,' | training loss:', training_loss, ' | test loss:', test_loss)
#     saved_path = saver.save(sess, '/content/drive/MyDrive/Colab Notebooks/result/nn_data/net', global_step=1000)
#     pickle.dump(sess, "saver")
    
    # # Use model to classify test data
    # classifications_on_test_data = sess.run(model, feed_dict={X: X_test})
    
    # # Check model accuracy
    # classes = (classifications_on_test_data > .5).astype(int)
    
    # print('\nAccuracy Score:',metrics.accuracy_score(y_test, classes))
    
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({X: X_test, y: y_test}))
    no_sig_predictions = tf.equal(tf.argmax(model1, 1), tf.argmax(y, 1))
    no_sig_accuracy = tf.reduce_mean(tf.cast(no_sig_predictions, "float"))
    print('Accuracy without sigmoid: ', no_sig_accuracy.eval({X: X_test, y: y_test}))

2022-03-05 17:10:18.830889: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-05 17:10:18.830917: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-05 17:10:18.830933: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thuy-ThinkPad-T480): /proc/driver/nvidia/version does not exist
2022-03-05 17:10:18.831117: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


epoch: 0  | training loss: 1.2439755  | test loss: 0.7964383
epoch: 100  | training loss: 0.0072227735  | test loss: 0.18834677
epoch: 200  | training loss: 0.0005067854  | test loss: 0.15602176
epoch: 300  | training loss: 0.0002024181  | test loss: 0.113301605
epoch: 400  | training loss: 0.00010763543  | test loss: 0.12240543
epoch: 500  | training loss: 6.634898e-05  | test loss: 0.1292745
epoch: 600  | training loss: 4.4768163e-05  | test loss: 0.13479961
epoch: 700  | training loss: 3.210395e-05  | test loss: 0.13940835
epoch: 800  | training loss: 2.405548e-05  | test loss: 0.14337963
epoch: 900  | training loss: 1.86288e-05  | test loss: 0.14687671
Accuracy: 0.97902095
Accuracy without sigmoid:  0.97902095


In [12]:
#Train plain model
class train():
    def __init__(self):
        with tf.Session() as sess:
            saver.restore(sess, './nn_data/net-1000')
            with tf.variable_scope('dense1', reuse=True):
                self.dense1_kernel = sess.run(tf.get_variable('kernel'))
                self.dense1_bias = sess.run(tf.get_variable('bias'))
            with tf.variable_scope('dense2', reuse=True):
                self.dense2_kernel = sess.run(tf.get_variable('kernel'))
                self.dense2_bias = sess.run(tf.get_variable('bias'))
            with tf.variable_scope('dense3', reuse=True):
                self.dense3_kernel = sess.run(tf.get_variable('kernel'))
                self.dense3_bias = sess.run(tf.get_variable('bias'))
            self.encoded_input = X.eval({X: X_test})
    
    def forward(self, x):
        #dense1
        out = x.dot(self.dense1_kernel) + self.dense1_bias        
        #square
        out = out*out        
        #dense2
        out = out.dot(self.dense2_kernel) + self.dense2_bias
        #square
        out = out*out        
        #dense3
        out = out.dot(self.dense3_kernel) + self.dense3_bias
        
        return out
        

In [13]:
#train encrypted input
class train_Enc_Model():
    def __init__(self):
        with tf.Session() as sess:
            saver.restore(sess, './nn_data/net-1000')
            with tf.variable_scope('dense1', reuse=True):
                self.dense1_kernel = sess.run(tf.get_variable('kernel'))
                self.dense1_bias = sess.run(tf.get_variable('bias'))
            with tf.variable_scope('dense2', reuse=True):
                self.dense2_kernel = sess.run(tf.get_variable('kernel'))
                self.dense2_bias = sess.run(tf.get_variable('bias'))
            with tf.variable_scope('dense3', reuse=True):
                self.dense3_kernel = sess.run(tf.get_variable('kernel'))
                self.dense3_bias = sess.run(tf.get_variable('bias'))
            self.encoded_input = X.eval({X: X_test})
    
    def forward(self, enc_x: ts.CKKSVector) -> ts.CKKSVector:
        #dense1
        out = enc_x.mm_(self.dense1_kernel) + self.dense1_bias        
        #square
        out = out.square_()        
        #dense2
        out = out.mm_(self.dense2_kernel) + self.dense2_bias        
        #square
        out = out.square_()        
        #dense3
        out = out.mm_(self.dense3_kernel) + self.dense3_bias
        
        return out
        

In [14]:
# encryption parameters
poly_mod_degree = 2 ** 14
power = 30
coeff_mod_bit_sizes = [40] + ([power] * 6) + [40]

# create TenSEALContext
ctx_eval = ts.Context(ts.SCHEME_TYPE.CKKS,poly_mod_degree,-1,coeff_mod_bit_sizes=coeff_mod_bit_sizes,)

# scale of ciphertext to use
ctx_eval.global_scale = 2 ** power

# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()

In [15]:
enc_predictions = []
plain_predictions = []

ckks_model = train_Enc_Model()
model = train()

for index in range(len(X_test)):
    #plain
    result= model.forward(X_test[index])
    result = np.argmax(result)
    plain_predictions.append(result)
    
    #ckks
    enc_sample = ts.ckks_vector(ctx_eval, X_test[index])
    enc_prediction = ckks_model.forward(enc_sample)
    dec_result = enc_prediction.decrypt(secret_key=ctx_eval.secret_key())
    
    dec_result = np.argmax(dec_result)
    enc_predictions.append(dec_result)
    
    

INFO:tensorflow:Restoring parameters from ./nn_data/net-1000
INFO:tensorflow:Restoring parameters from ./nn_data/net-1000


In [16]:
plain_predictions[:20]

[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]

In [17]:
enc_count = 0
count = 0
for i in range(len(y_test)):
    if enc_predictions[i] != np.argmax(y_test[i]):
        enc_count += 1
    if plain_predictions[i] != np.argmax(y_test[i]):
        count += 1
        
print("Plain model:================")
print("Different plain: ", count)
print("Accuracy: plain", 1 - count/len(y_test))

print("Encrypted model:==================")
print("Different ckks: ", enc_count)
print("Accuracy ckks: ", 1 - enc_count/len(y_test))


Plain model:================
Different plain:  1
Accuracy: plain 0.993006993006993
Encrypted model:==================
Different ckks:  3
Accuracy ckks:  0.9790209790209791


In [18]:
saver = tf.train.Saver()

plain_predict = tf.convert_to_tensor(plain_predictions, dtype=tf.int64)
enc_predict = tf.convert_to_tensor(enc_predictions, dtype=tf.int64)
# PRINT ACCURACIES
with tf.Session() as sess:
    saver.restore(sess, './nn_data/net-1000')
    
    tf_guessed_predictions = tf.equal(plain_predict, tf.argmax(y, 1))
    tf_accuracy = tf.reduce_mean(tf.cast(tf_guessed_predictions, tf.float32))

    cn_guessed_predictions = tf.equal(enc_predict, tf.argmax(y, 1))
    cn_accuracy = tf.reduce_mean(tf.cast(cn_guessed_predictions, tf.float32))

    print("Accuracy plain: ", tf_accuracy.eval({X: X_test, y: y_test}))
    print("Accuracy CKKS: ", cn_accuracy.eval({X: X_test, y: y_test}))

INFO:tensorflow:Restoring parameters from ./nn_data/net-1000
Accuracy plain:  0.993007
Accuracy CKKS:  0.97902095


In [19]:
# tensor_predictions = []
# train_tensor = train_Enc_tensor()

# encrypted_tensor = ts.ckks_tensor(ctx_eval, X_test)
# print(encrypted_tensor)

In [14]:
with tf.Session() as sess:
    saver.restore(sess, './nn_data/net-1000')
    with tf.variable_scope('dense1', reuse=True):
        dense1_kernel = sess.run(tf.get_variable('kernel'))
        dense1_bias = sess.run(tf.get_variable('bias'))
    with tf.variable_scope('dense2', reuse=True):
        dense2_kernel = sess.run(tf.get_variable('kernel'))
        dense2_bias = sess.run(tf.get_variable('bias'))
    with tf.variable_scope('dense3', reuse=True):
        dense3_kernel = sess.run(tf.get_variable('kernel'))
        dense3_bias = sess.run(tf.get_variable('bias'))

INFO:tensorflow:Restoring parameters from ./nn_data/net-1000


In [16]:
np.save("./ckks_data/dense1_kernel", dense1_kernel)
np.save("./ckks_data/dense1_bias", dense1_kernel)
np.save("./ckks_data/dense2_kernel", dense2_kernel)
np.save("./ckks_data/dense2_bias", dense2_kernel)
np.save("./ckks_data/dense3_kernel", dense3_kernel)
np.save("./ckks_data/dense3_bias", dense3_kernel)